# Final Cleaning

In [1]:
%load_ext autoreload
%autoreload 2

#system
import os
import sys
from os.path import join as pj
module_path = os.path.abspath(pj('..','..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# utils
from src.d00_utils import print_helper_functions as phf

# ipython
import warnings
warnings.simplefilter('ignore')

# executing code
import click
import logging

# type annotations
from typing import List, Set, Dict, Tuple, Optional
from collections.abc import Iterable

# configuring
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
import configparser

# feature extractor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline, make_union, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer

# data
import numpy as np
import re
from forex_python.converter import CurrencyRates
np.set_printoptions(precision=4)
from dateutil.relativedelta import relativedelta
import datetime

# stats
import statsmodels.api as sm

# viz
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set(font_scale=1.5)
plt.style.use('bmh')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# file handling
import tempfile
import joblib
import botocore
import boto3
from os.path import join as pj
import pickle as pkl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1.0 Load data

In [2]:
# not used in this stub but often useful for finding various files
project_dir = Path().resolve().parents[1]
print(project_dir)

# find .env automagically by walking up directories until it's found, then
# load up the .env entries as environment variables
_ = load_dotenv(find_dotenv())

raw_dir = pj(project_dir, 'data', os.environ.get('RAW_DIR'))
interim_dir = pj(project_dir, 'data', os.environ.get('INTERIM_DIR'))
processed_dir = pj(project_dir, 'data', os.environ.get('PROCESSED_DIR'))

/Users/marclawson/repositories/grantnav_10k_predictor


In [3]:
data = pd.read_csv(pj(interim_dir,'grantnav_data_post2015.csv'), index_col=0)

## 2.0 Checks

### 2.1 Currency

In [4]:
data['currency'].value_counts()

GBP    319016
USD       479
EUR        24
CHF         4
CAD         2
ILS         1
Name: currency, dtype: int64

In [5]:
# load currencies conversions for award date

In [6]:
nongbp_currencies = data[data['currency']!='GBP'].sort_values('award_date')

In [7]:
currencies_dict = dict(zip(nongbp_currencies.award_date, nongbp_currencies.currency))

In [11]:
import requests

In [21]:
api = '2c81b56d5ca58364ea22'

In [42]:
example_date = list(currencies_dict.keys())[0]
example_currency = list(currencies_dict.values())[0]

In [17]:
example_currency
example_date

'2015-01-20'

'USD'

In [55]:
today = datetime.datetime.now()-relativedelta(years=1)
today_last_year = today.strftime('%Y-%m-%d')

In [56]:
URL = f'https://free.currconv.com/api/v7/convert?q={example_currency}_GBP,GBP_{example_currency}&compact=ultra&date={today_last_year}&apiKey={api}'
print(URL)
req_ = requests.get(URL, params=api)
json_ = req_.json()
print(json_)

https://free.currconv.com/api/v7/convert?q=USD_GBP,GBP_USD&compact=ultra&date=2020-06-03&apiKey=2c81b56d5ca58364ea22
{'USD_GBP': {'2020-06-03': 0.795295}, 'GBP_USD': {'2020-06-03': 1.257395}}


In [57]:
URL_usage = f'https://free.currconv.com/others/usage?apiKey={api}'
params = {'api_key': api} 
req_ = requests.get(URL_usage, params=api)
json_ = req_.json()
print(json_)

{'timestamp': '2021-06-03T14:49:48.359Z', 'usage': 10}


In [15]:
forex_dict = {}
for k, v in currencies_dict.items():
    URL = f'https://www.currencyconverterapi.com/api/v7/convert?q={v}_PHP,PHP_GBP&compact=ultra&date=[{k}]&apiKey={api}'
    r = requests.get(URL, 'text')

KeyboardInterrupt: 